# Subquery Exercises

<hr style="border:2px solid gray">

1. Find all the current employees with the same hire date as employee 101010 using a subquery.

2. Find all the titles ever held by all current employees with the first name Aamod.

3. How many people in the employees table are no longer working for the company? Give the answer in a comment in your code.

4. Find all the current department managers that are female. List their names in a comment in your code.

5. Find all the employees who currently have a higher salary than the companies overall, historical average salary.

6. How many current salaries are within 1 standard deviation of the current highest salary? (Hint: you can use a built in function to calculate the standard deviation.) What percentage of all salaries is this?
    - Hint You will likely use multiple subqueries in a variety of ways
    - Hint It's a good practice to write out all of the small queries that you can. Add a comment above the query showing the number of rows returned. You will use this number (or the query that produced it) in other, larger queries.

<b> Subquery BONUS</b>

1. Find all the department names that currently have female managers.
2. Find the first and last name of the employee with the highest salary.
3. Find the department name that the employee with the highest salary works in.
4. Who is the highest paid employee within each department.

<hr style="border:1px solid black">
<hr style="border:1px solid black">

### Exercises

<b>1. Find all the current employees with the same hire date as employee 101010 using a subquery.

In [ ]:
SELECT *
FROM employees
JOIN dept_emp USING(emp_no)
WHERE to_date > now()
AND hire_date =
    (SELECT hire_date
    FROM employees
    WHERE emp_no = 101010);

In [ ]:
#check our work
SELECT hire_date
FROM employees
WHERE emp_no = 101010;

<div class="alert alert-block alert-info">
<b>Note:</b>
    
One way that we can check our work is to run that subquery alone and determine what the actual hire date for employee #101010 is. 

Employee #101010's hire date is 1990-10-22

So, if the query above gives us all the same hire dates as that employee... we are good to go!
    
</div>

<hr style="border:0.5px solid grey">

<b>2. Find all the titles ever held by all current employees with the first name Aamod.

In [ ]:
SELECT DISTINCT title
FROM titles
WHERE emp_no IN (
   SELECT emp_no
   FROM employees
   JOIN dept_emp USING(emp_no)
   WHERE first_name = 'aamod'
   AND to_date > now()
);

<hr style="border:0.5px solid grey">

<b>3. How many people in the employees table are no longer working for the company? Give the answer in a comment in your code.

In [ ]:
SELECT count(*) # count all the records
FROM employees # from the employees table
WHERE emp_no NOT IN 
    (SELECT emp_no
    FROM dept_emp
    WHERE to_date > NOW()
);

In [ ]:
##Same solution using 'salaries' table
SELECT count(*)
FROM employees
WHERE emp_no NOT IN 
    (SELECT emp_no
    FROM salaries
    WHERE to_date > now()
);

<b>Answer</b>:  59,900

<div class="alert alert-block alert-info">
<b>Note:</b>
    
We can use employees table or salaries table in our subquery because they both give us to_date. By saying NOT IN we are telling SQL that those are employees that no longer work for that company.

<hr style="border:0.5px solid grey">

<b>4. Find all the current department managers that are female. List their names in a comment in your code.

In [ ]:
SELECT *
FROM employees
WHERE emp_no in (
    SELECT emp_no
       FROM dept_manager
          WHERE to_date > now()
)
AND gender = "F";

<b>Answer</b>: 
- Isamu Legleitner
- Karsten Sigstam
- Leon DasSarma
- Hilary Kambil

<hr style="border:0.5px solid grey">

<b>5. Find all the employees who currently have a higher salary than the companies overall, historical average salary.

In [ ]:
SELECT emp_no, salary
FROM salaries s
JOIN employees e using(emp_no)
WHERE to_date > now() 
AND  salary > (SELECT AVG(salary) FROM salaries);

In [ ]:
#double check by seeing what avg salary is
SELECT Round(AVG(salary),0) 
FROM salaries;

<b>Answer</b>: 154,543 employees who currently have a higher salary then company average.

<div class="alert alert-block alert-info">
<b>Note:</b>
    
One way that we can check our work is to find the overall, historic average salary.

Average salary was $63,811

So, if the query above gives us salaries above that, we are on the right track!
    
</div>

<hr style="border:0.5px solid grey">

<b>6. How many current salaries are within 1 standard deviation of the current highest salary? (Hint: you can use a built in function to calculate the standard deviation.) What percentage of all salaries is this?</b>
   - Hint: You will likely use multiple subqueries in a variety of ways
   - Hint: It's a good practice to write out all of the small queries that you can. Add a comment above the query showing the number of rows returned. You will use this number (or the query that produced it) in other, larger queries.

In [ ]:
#First, what is the max current salary
SELECT MAX(salary) 
FROM salaries 
WHERE to_date > now();

# $158,220 is the max current salary

In [ ]:
#Second, what is 1 std for current salary
SELECT STDDEV(salary) 
FROM salaries 
WHERE to_date > now();

# $17,309 is one STD from the salary

In [ ]:
#Count of current salaries > (Max-1 std). (count = 83)
SELECT count(*)
FROM salaries
WHERE to_date > now()
AND salary > (
(SELECT MAX(salary) FROM salaries WHERE to_date > now()) - 
(SELECT STDDEV(salary) FROM salaries WHERE to_date > now())
);

<div class="alert alert-block alert-info">
<b>Note:</b>
    
Here we are saying: give me a count for all the <u>current</u> salaries where the salary is <u>greater</u> than the max salary (158K) minus the std (17K).
    
So, a count of employees who make more than 141K
</div>

In [ ]:
#Denominator, count of all current salaries (240,124)
SELECT count(*)
FROM salaries 
WHERE to_date > now();

In [ ]:
#What percentage of all salaries is this?
#SELECT (Numerator-83)/ (Denominator-240124)

SELECT((SELECT count(*)
FROM salaries
WHERE to_date > now()
AND salary > (
(SELECT MAX(salary) FROM salaries WHERE to_date > now()) - 
(SELECT STD(salary) FROM salaries WHERE to_date > now())
))/(SELECT count(*)
FROM salaries 
WHERE to_date > now())) * 100 AS "percentage of salaries within 1 Stdev of Max";

<hr style="border:1px solid black">

### Subquery BONUS Exercises

<b>1. Find all the department names that currently have female managers.

In [ ]:
SELECT dept_name
FROM departments
WHERE dept_no IN (
  SELECT dept_no
  FROM dept_manager
  WHERE emp_no IN (
    SELECT emp_no
    FROM employees
    WHERE gender = 'F'
  )
  AND to_date > NOW()
);

<hr style="border:0.5px solid grey">

<b>2. Find the first and last name of the employee with the highest salary.

In [ ]:
SELECT first_name, last_name
FROM employees
WHERE emp_no IN (
  SELECT emp_no
  FROM salaries
  WHERE salary = (
    SELECT MAX(salary)
    FROM salaries
  )
);

<hr style="border:0.5px solid grey">

<b>3. Find the department name that the employee with the highest salary works in.

In [ ]:
SELECT d.dept_name
FROM departments d
JOIN dept_emp de ON d.dept_no = de.dept_no
JOIN (
  SELECT emp_no
  FROM salaries
  WHERE salary = (
    SELECT MAX(salary)
    FROM salaries
  )
) s ON de.emp_no = s.emp_no
LIMIT 1;

<hr style="border:0.5px solid grey">

<b>4. Who is the highest paid employee within each department.

In [ ]:
SELECT d.dept_name, e.first_name, e.last_name, s.salary
FROM departments d
    JOIN dept_emp de USING(dept_no)
    JOIN employees e USING(emp_no)
    JOIN salaries s USING(emp_no)
WHERE s.salary IN (
  SELECT MAX(salary)
  FROM salaries
  GROUP BY d.dept_no
)
ORDER BY d.dept_name, s.salary DESC;

<hr style="border:0.5px solid grey">

<hr style="border:0.5px solid grey">